<a href="https://colab.research.google.com/github/petermr/semanticClimate/blob/main/abbreviation/Codes/abbreviation_extraction_revamped.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install abbreviations
!pip install scispacy==0.3.0
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 2.0 MB/s 
     |████████████████████████████████| 10.4 MB 47.5 MB/s 
     |████████████████████████████████| 13.5 MB 36.5 MB/s 
     |████████████████████████████████| 71 kB 9.8 MB/s 
     |████████████████████████████████| 188 kB 63.7 MB/s 
     |████████████████████████████████| 1.0 MB 59.9 MB/s 
     |████████████████████████████████| 184 kB 68.5 MB/s 
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.3
    Uninstalling srsly-2.4.3:
      Successfully uninstalled srsly-2.4.3
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.7
    Uninstalling catalogue-2.0.7:
      Successfully uninstalled catalogue-2.0.7
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninst

In [ ]:
!git clone  https://github.com/petermr/semanticClimate.git

Cloning into 'semanticClimate'...
remote: Enumerating objects: 1261, done.
remote: Counting objects: 100% (313/313), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 1261 (delta 118), reused 222 (delta 71), pack-reused 948
Receiving objects: 100% (1261/1261), 138.81 MiB | 36.54 MiB/s, done.
Resolving deltas: 100% (199/199), done.


In [ ]:
import spacy
import scispacy
from bs4 import BeautifulSoup
from abbreviations import schwartz_hearst
from scispacy.abbreviation import AbbreviationDetector
from IPython.display import HTML
import pandas as pd
import requests
import os
dict_abbreviation = {}

class abbreviation_extraction():
  def __init__(self, html_path, saving_path, model):
    self.html_path = html_path
    self.saving_path = saving_path
    self.dict_abbreviation = {}
    self.dict_abbreviation_swartz = {}
    self.text = ''

  def make_clickable(val):
          #print(type(val))
          l = [f'<a target="_blank" href="{val_}">{val_}</a>' for val_ in val]
          return l
  def wikidata_lookup(self, row):
          query = row['long_form']
          #print('query',query)
          params	= {
                  "action"		: "wbsearchentities",
                  "search"		: query,
                  "language"	: "en",
                  "format"		: "json"
                }
          data	= requests.get("https://www.wikidata.org/w/api.php",params=params)
          result = data.json()
          hit_list = []
          for hit in result['search']:
            try:
              if "scientific article" not in hit["description"]:
                  hit_list.append(hit["url"])
            except:
                  hit_list.append(hit["url"])
          return hit_list
  
  def text_extraction(self):
    with open(self.html_path, 'r') as f:
      html = f.read()
      soup = BeautifulSoup(html, features="html.parser")

      # kill all script and style elements
      for script in soup(["script", "style"]):
          script.extract()    # rip it out

      # get text
      text = soup.get_text()

      # break into lines and remove leading and trailing space on each
      lines = (line.strip() for line in text.splitlines())
      # break multi-headlines into a line each
      chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
      # drop blank lines
      #text_write = '\n'.join(chunk for chunk in chunks if chunk)
      self.text = '\n'.join(chunk for chunk in chunks if chunk)
      #print(text)
    return self.text

  def scispacy_extraction(self, doc):
    for abrv in doc._.abbreviations:
        new_dict  ={}
        abrv_ = str(abrv)
        long_f = str(abrv._.long_form)
        if '(' in  long_f or ')' in long_f or abrv_ == long_f or abrv_ == long_f +'s':
            continue
        else:  
            if abrv_ not in self.dict_abbreviation:
              new_dict['long_form']= abrv._.long_form
              new_dict['count'] = int(1)
              self.dict_abbreviation[str(abrv)] = new_dict  
            else:  
              self.dict_abbreviation[str(abrv)]['count'] = int(self.dict_abbreviation[str(abrv)]['count'])+ 1
              continue
    return self.dict_abbreviation

  def dict_extraction_swartz(self, pairs):
    for key, value in pairs.items():
    
      new_dict  ={}
      abrv_ = key
      long_f = value
      if '(' in  long_f or ')' in long_f or abrv_ == long_f or abrv_ == long_f +'s':
          continue
      else:  
          if abrv_ not in self.dict_abbreviation_swartz:
            new_dict['long_form']= long_f
            new_dict['count'] = int(1)
            self.dict_abbreviation_swartz[str(abrv_)] = new_dict  
          else:  
            self.dict_abbreviation_swartz[str(abrv_)]['count'] = int(self.dict_abbreviation_swartz[str(abrv_)]['count'])+ 1
            continue
    return self.dict_abbreviation_swartz  
  
  def table_generation(self, dictionary_abbreviation,CSV_SAVE_PATH):
        #CSV_SAVE_PATH = self.saving_path + 'abbreviation_table_.csv'
        entry = {'abbreviation':[''],'long_form':[''],'count':[0]}
        df = pd.DataFrame(entry)
        for l in dictionary_abbreviation:
          #print(l)
          entry = {'abbreviation':str(l),'long_form':str(dictionary_abbreviation[l]['long_form']),'count':dictionary_abbreviation[l]['count']}
          df = df.append(entry, ignore_index= True)
        df = df.drop(0)
        df = df.reset_index(drop=True)
        df_copy = df.copy()
        #print(df) 

        df['wiki_search_links'] = df.apply(lambda row: self.wikidata_lookup(row), axis =1)
        

        df.style.format({'wiki_search_links': self.make_clickable})      
        df.to_csv(CSV_SAVE_PATH, index = None)
  def main(self):
      
      if not os.path.exists(self.saving_path):
        os.makedirs(self.saving_path) 
      
      text = self.text_extraction()
      
      pairs = schwartz_hearst.extract_abbreviation_definition_pairs(doc_text= text)
      #print('pairs',pairs)
      
      TEXT_ = f'Chapter_text.txt'
      DICTIONARY_SH = f'Chapter_dictionary_SH.txt'
      DICTIONARY_Spacy= f'Chapter_dictionary_Spacy.txt'
      with open(self.saving_path + TEXT_, 'w') as file:
          file.write(text)
      
      with open(self.saving_path + DICTIONARY_SH, 'w') as file:
          for key, value in pairs.items():
            #print(key, value )
            file.write('{'+str(key)+' : '+ str(value)+"}")
            file.write('\n')
      

      nlp = spacy.load('en_core_web_sm')
      abbreviation_pipe = AbbreviationDetector(nlp)
      nlp.add_pipe(abbreviation_pipe)
      doc = nlp(text)
      #print(doc._._)
      #print(doc._.abbreviations)
      
      with open(self.saving_path + DICTIONARY_Spacy, 'w') as file:
          for abrv in doc._.abbreviations:
            
            #print(abrv ,abrv._.long_form)
            #print(key, value )
            file.write('{'+str(abrv)+' : '+ str(abrv._.long_form)+"}")  
            file.write('\n')
      

      self.scispacy_extraction(doc)
      self.dict_extraction_swartz(pairs)
      CSV_SAVE_PATH = self.saving_path + 'abbreviation_table_spacy.csv'
      #dictionary_abbreviation = self.dict_abbreviation
      dictionary_abbreviation = self.dict_abbreviation
      self.table_generation(dictionary_abbreviation,CSV_SAVE_PATH)
      
      #return df

In [ ]:

html_path = '/content/semanticClimate/ipcc/ar6/wg3/Chapter06/fulltext.flow.html'
saving_path = '/content/'
model = ''

abbreviation = abbreviation_extraction(html_path, saving_path, model)
abbreviation.main()

In [ ]:
#analysis:
df_spacy = pd.read_csv('/content/abbreviation_table_spacy.csv')
df_swartz = pd.read_csv('/content/abbreviation_table_swartz.csv')
print(f'Length of the df_spacy : {df_spacy.shape[0]} \nLength of the df_swartz : {df_swartz.shape[0]}\n')
intersection = set(list(df_swartz.abbreviation.to_numpy())).intersection(list(df_spacy.abbreviation.to_numpy()))
print(f'names of the abbr common in abbreviation from scpay and swartz: \n{list(intersection)}\n')
print(f'Length of the intersection: {len(list(intersection))}')

Length of the df_spacy : 60 
 Length of the df_swartz : 41

names of the abbr common in abbreviation from scpay and swartz: 
['PERC', 'SMR', 'LCOE', 'LCA', 'FIT', 'ATB', 'LOHCs', 'PRIS', 'SDGs', 'TPES', 'OLEDs', 'CROI', 'IEA', 'CDR', 'LFAC', 'BECCS', 'LABs', 'LAM', 'DEV', 'VRE', 'CSP', 'LNG', 'PVSC', 'IoT', 'PtL', 'CCS', 'TFC', 'KEBA', 'HTL', 'ETS', 'CDU', 'EPCC', 'MTES', 'LPG', 'EROI']

Length of the intersection: 35


In [ ]:
union = set(list(df_swartz.abbreviation.to_numpy())).union(list(df_spacy.abbreviation.to_numpy()))
print(f'names of the abbr all together in abbreviation from scpay and swartz: \n{list(intersection)}\n')
print(f'Length of the union: {len(list(union))}')

names of the abbr all together in abbreviation from scpay and swartz: 
['PERC', 'SMR', 'LCOE', 'LCA', 'FIT', 'ATB', 'LOHCs', 'PRIS', 'SDGs', 'TPES', 'OLEDs', 'CROI', 'IEA', 'CDR', 'LFAC', 'BECCS', 'LABs', 'LAM', 'DEV', 'VRE', 'CSP', 'LNG', 'PVSC', 'IoT', 'PtL', 'CCS', 'TFC', 'KEBA', 'HTL', 'ETS', 'CDU', 'EPCC', 'MTES', 'LPG', 'EROI']

Length of the union: 66
